# Обучение модели (BERT) классификации комментариев

Интернет-магазин «Y» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в Y-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В распоряжении набор данных с разметкой о токсичности.
Требование бизнеса - модель со значением метрики качества F1 не меньше 0.75.

## Загрузка данных, библиотек

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from sklearn.metrics import f1_score

import torch
from torch.nn.utils.rnn import pad_sequence
import transformers as ppb
import os
import warnings
warnings.filterwarnings('ignore')

import re
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer 
nltk.download("stopwords")
from nltk.corpus import stopwords as nltk_stopwords

from tqdm import tqdm
tqdm.pandas()
from tqdm import notebook 

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# просмотр, где находится каталог с файлами на COLAB
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# получение доступа к каталогу и уточнение названия папок
import os
os.listdir('/content/drive/My Drive/Colab Notebooks/Яндекс/Проект 13 «Викишоп»/ДАННЫЕ')

['toxic_comments.csv', 'cased_L-12_H-768_A-12', 'rndlr96_EnBERT']

In [ ]:
# загрузка данных
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Яндекс/Проект 13 «Викишоп»/ДАННЫЕ/toxic_comments.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [ ]:
data.duplicated().sum() # подсчёт явных дубликатов

0

### Вывод

**В результате загрузки данных, установлено:**

1.	DataFrame содержит 159571 строк и 2 столбца.
2.	В столбце «text» данные типа object, пропуски отсутствуют.
3.	В столбце «toxic» данные типа int64, пропуски отсутствуют.
4.	Явные дубликаты отсутствуют.


## Предобработка данных

#### Очистка данных

In [ ]:
data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [ ]:
# функция лемматизации
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    lemm_list = nltk.word_tokenize(text)
    lemm_text = " ".join([lemmatizer.lemmatize(l) for l in lemm_list])     
    return lemm_text

In [ ]:
# функция очистки
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    return " ".join(text.split())

In [ ]:
data['clear_text'] = data['text'].progress_apply(clear_text)
data['clear_text'] = data['clear_text'].progress_apply(lemmatize)

100%|██████████| 159571/159571 [01:51<00:00, 1435.24it/s]


### light выборка

*Так как процесс моделирования на всем датафреме достаточно ресурсозатратный, выберем случайным образом 500 строк и обучим на них модель*

In [ ]:
df = data.sample(500, random_state=23031998).reset_index(drop=True)

### Проверим баланс классов

In [ ]:
print("Соотношение класса '0' и '1' соответственно:",
      round(df[df["toxic"] == 0].shape[0] / (df[df["toxic"] == 0].shape[0] + df[df["toxic"] == 1].shape[0]), 2), ":",
      round(df[df["toxic"] == 1].shape[0] / (df[df["toxic"] == 0].shape[0] + df[df["toxic"] == 1].shape[0]), 2))

Соотношение класса '0' и '1' соответственно: 0.88 : 0.12


### Вывод

**В результате предобработки:**

1. Произведена очистка данных и лемматизация (для BERT можно и не делать лемматизацию, она и так хорошо справляется).
2. Выбрано случайным образом 500 строк, ввиду ресурсоёмкости обработки всего массива данных.
3. Установлено, что соотношение класса '0' и '1' соответственно: 0.88 : 0.12, данный факт указывает на необходимость балансировки, однако, для начала необходимо подготовить модели.

#### Загрузка предобученной BERT

In [ ]:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased') #данная модель работает с длинной токена 512


In [ ]:
# Загрузка предобученной модели/токенизатора 
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenized = df['clear_text'].progress_apply(
    lambda x: tokenizer.encode(x, max_length=512, truncation=True, add_special_tokens=True)) #обрезаем под нужную длину токена

padded = pad_sequence([torch.as_tensor(seq) for seq in tokenized], batch_first=True) #выравниваем длину нулями  

attention_mask = padded > 0
attention_mask = attention_mask.type(torch.LongTensor)

100%|██████████| 500/500 [00:01<00:00, 451.46it/s]


#### features & target

In [ ]:
%%time
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

features = np.concatenate(embeddings) 

  0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 14min 59s, sys: 10.3 s, total: 15min 10s
Wall time: 15min 12s


In [ ]:
features_train, features_test, target_train, target_test = train_test_split(features, df['toxic'], test_size=0.25)

### Вывод

**В результате подготовки модели, а также features & target:**

1.  Загружена предобученная модель `BertModel`.
2.  Выполнена токенизация и кодирование строк.
3.  Выполнена подрезка длинны токенов (не должно превышать 512 (обусловлено особенностями используемой модели BERT) 
4.  Выборка разделена на тренировочную и тестовую (75:25).


## Моделирование

### Поиск гиперпараметров

####  LogisticRegression

In [ ]:
# функция поиска best_score и параметров модели LogisticRegression
def LogisticRegression_model(features_train, target_train):
  model = LogisticRegression()
  parametrs = { 'C': range(10, 30, 1),
               'class_weight':['balanced', None] }
  search = HalvingGridSearchCV(model, parametrs, cv=5, scoring='f1')
  search.fit(features_train, target_train)
  best_model_LogisticRegression = search.best_estimator_
  best_score_model_LogisticRegression = round(search.best_score_, 3)
  
  return best_model_LogisticRegression, best_score_model_LogisticRegression

In [ ]:
best_model_LogisticRegression, best_score_model_LogisticRegression = LogisticRegression_model(features_train, target_train)

#### CatBoostClassifier

In [ ]:
# функция поиска best_score и параметров модели CatBoostClassifier
def CatBoostClassifier_model(features_train, target_train):
  model = CatBoostClassifier()
  parametrs = { 'depth': range (1, 3, 1),
              'n_estimators': range (1, 10, 1) }
  search = HalvingGridSearchCV(model, parametrs, cv=5, scoring='f1')
  search.fit(features_train, target_train)
  best_model_CatBoostClassifier = search.best_estimator_
  best_score_model_CatBoostClassifier = round(search.best_score_, 3)
  
  return best_model_CatBoostClassifier, best_score_model_CatBoostClassifier

In [ ]:
best_model_CatBoostClassifier, best_score_model_CatBoostClassifier = CatBoostClassifier_model(features_train, target_train)

Learning rate set to 0.5
0:	learn: 0.4569468	total: 6.56ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.2084242	total: 3.1ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3874567	total: 3.1ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3874567	total: 3.05ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.4505279	total: 3.04ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.4569468	total: 3.14ms	remaining: 3.14ms
1:	learn: 0.3628890	total: 6.67ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.2084242	total: 3.05ms	remaining: 3.05ms
1:	learn: 0.0683496	total: 6.55ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3874567	total: 3.01ms	remaining: 3.01ms
1:	learn: 0.1749870	total: 6.35ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3874567	total: 3.05ms	remaining: 3.05ms
1:	learn: 0.1723719	total: 6.42ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.4505279	total: 3.1ms	remaining: 3.1ms
1:	learn: 0.3741724	total: 6.48ms	remaining: 0us
Lear

#### RandomForestClassifier

In [ ]:
# функция поиска best_score и параметров модели RandomForestClassifier
def RandomForestClassifier_model(features_train, target_train):
  model = RandomForestClassifier()
  parametrs = { 'max_depth': range (6, 12, 1),
              'n_estimators': range (25, 30, 1) }
  search = HalvingGridSearchCV(model, parametrs, cv=5, scoring='f1')
  search.fit(features_train, target_train)
  best_model_RandomForestClassifier = search.best_estimator_
  best_score_model_RandomForestClassifier = round(search.best_score_, 3)

  return best_model_RandomForestClassifier, best_score_model_RandomForestClassifier

In [ ]:
best_model_RandomForestClassifier, best_score_model_RandomForestClassifier = RandomForestClassifier_model(features_train, target_train)

#### LGBMClassifier

In [ ]:
# функция поиска best_score и параметров модели LGBMClassifier
def LGBMClassifier_model(features_train, target_train):
  model = LGBMClassifier()
  parametrs = { 'max_depth': range (1, 5, 1),
              'n_estimators': range (1, 10, 1) }
  search = HalvingGridSearchCV(model, parametrs, cv=5, scoring='f1')
  search.fit(features_train, target_train)
  best_model_LGBMClassifier = search.best_estimator_
  best_score_model_LGBMClassifier = round(search.best_score_, 3)
  
  return best_model_LGBMClassifier, best_score_model_LGBMClassifier

In [ ]:
best_model_LGBMClassifier, best_score_model_LGBMClassifier = LGBMClassifier_model(features_train, target_train)

### Рейтинг моделей по метрике качества (F1)

In [ ]:
list_model = [best_model_LogisticRegression,
              best_model_CatBoostClassifier,
              best_model_RandomForestClassifier,
              best_model_LGBMClassifier]
              
list_score = [best_score_model_LogisticRegression,
              best_score_model_CatBoostClassifier,
              best_score_model_RandomForestClassifier,
              best_score_model_LGBMClassifier]

In [ ]:
intermediate_dictionary = {'Model':list_model, 'F1':list_score}
rating_model = pd.DataFrame(intermediate_dictionary)
rating = rating_model.sort_values(by='F1', ascending=False)
rating

,Model,F1
0,"LogisticRegression(C=18, class_weight='balanced')",0.444
1,<catboost.core.CatBoostClassifier object at 0x...,0.303
2,"(DecisionTreeClassifier(max_depth=6, max_featu...",0.167
3,"LGBMClassifier(max_depth=2, n_estimators=6)",0.000


In [ ]:
print('Первое место занимает', rating.iloc[0,0])
print('Значении F1 равно', rating.iloc[0,1])

Первое место занимает LogisticRegression(C=18, class_weight='balanced')
Значении F1 равно 0.444


In [ ]:
best_model_on_data_default = rating.iloc[0,0]

### Вывод

**В результате обучения и подбора лучших параметров моделей default данных, установлено:**

1. Лучшая модель: LogisticRegression.
2. Параметры лучшей модели: C=18, class_weight='balanced'.
3. Качество модели (F1): 0.444.
4. Не высокое качество модели обусловлено дисбалансом классов.

## Поиск множителя балансировки, а также наиболее оптимальной модели

In [ ]:
target_train = pd.DataFrame(target_train)
features_train = pd.DataFrame(features_train)

In [ ]:
train = np.concatenate((features_train, target_train), axis=1)
train = pd.DataFrame(train)

In [ ]:
print("Соотношение класса '0' и '1' соответствено:",
      round(train[train.iloc[:,-1] == 0].shape[0] / (train[train.iloc[:,-1] == 0].shape[0] + train[train.iloc[:,-1] == 1].shape[0]), 2), ":",
      round(train[train.iloc[:,-1] == 1].shape[0] / (train[train.iloc[:,-1] == 0].shape[0] + train[train.iloc[:,-1] == 1].shape[0]), 2))

Соотношение класса '0' и '1' соответствено: 0.88 : 0.12


In [ ]:
# функция увеличения выборки
def upsample(dataset, repeat):
  train_zeros = dataset[dataset.iloc[:,-1] == 0]
  train_ones = dataset[dataset.iloc[:,-1] == 1]
  train_upsampled = pd.concat([train_zeros] + [train_ones] * repeat)
  train_up = shuffle(
        train_upsampled, random_state=12345)
  return train_up

In [ ]:
# функция поиска наиболее оптимального модели, а также значения МНОЖИТЕЛЯ (хN), best_score и параметров модели
def search_best_xN_F1(dataset, n, m):
  best_xN = 0
  best_F1 = 0
  best_model = None
  best_target_up = None
  best_features_up = None
  for j in range(1, 5):
    train_up = upsample(dataset, j)
    target_up = train_up.iloc[:,-1]
    features_up = train_up.iloc[:,:-1]

    li_model = [LogisticRegression_model(features_up, target_up),
                CatBoostClassifier_model(features_up, target_up),
                RandomForestClassifier_model(features_up, target_up),
                LGBMClassifier_model(features_up, target_up)]
    for mod in li_model:
      model, result = mod
      if result > best_F1:
        best_F1 = result
        best_xN = j
        best_model = model

  return best_xN, best_F1, best_model, best_target_up, best_features_up

In [ ]:
%%time
best_xN, best_F1, best_model, best_target_up, best_features_up = search_best_xN_F1(train, 1, 5)

Learning rate set to 0.5
0:	learn: 0.3874567	total: 5.38ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.4887235	total: 3.13ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3769760	total: 3.06ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3769760	total: 3.06ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.4887235	total: 3.09ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3874567	total: 3.3ms	remaining: 3.3ms
1:	learn: 0.2072654	total: 6.7ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.4887235	total: 9.71ms	remaining: 9.71ms
1:	learn: 0.2989211	total: 14.4ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3769760	total: 3.19ms	remaining: 3.19ms
1:	learn: 0.1742880	total: 7.34ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3769760	total: 3.04ms	remaining: 3.04ms
1:	learn: 0.1814913	total: 6.67ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.4887235	total: 3.01ms	remaining: 3.01ms
1:	learn: 0.2964019	total: 6.28ms	remaining: 0us
Lea

In [ ]:
print("Лучший множитель:", best_xN)
print("Лучший f1_score:", round(best_F1, 2))
print("Лучшая модель:", best_model)

Лучший множитель: 4
Лучший f1_score: 0.86
Лучшая модель: RandomForestClassifier(max_depth=8, n_estimators=25)


In [ ]:
best_model_on_data_custom = best_model

### Вывод

**В результате поиска множителя балансировки, а также подбора лучших параметров моделей на custom данных, установлено:**

1. Высокое качество модели RandomForestClassifier(max_depth=8, n_estimators=25) достигается на множителе - 4.
2. Значение F1: 0.86.
3. Время поиска множителя для балансировки и параметров модели порядка: 8 мин.

## Проверка качества модели на тестовой выборке

### best_model_on_data_default

In [ ]:
best_model_on_data_default.fit(features_train, target_train)
test_predictions = best_model_on_data_default.predict(features_test)
result = f1_score(target_test, test_predictions)
print("F1 лучшей модели", best_model_on_data_default, "на тестовой выборке:", abs(round(result, 2)))

F1 лучшей модели LogisticRegression(C=18, class_weight='balanced') на тестовой выборке: 0.52


### best_model_on_data_custom

In [ ]:
features = pd.DataFrame(features)
target = pd.DataFrame(df['toxic'])

In [ ]:
dat = np.concatenate((features, target), axis=1)
dat = pd.DataFrame(dat)
dat = shuffle(
        dat, random_state=23031998)


In [ ]:
dat_up = upsample(dat, 4)

In [ ]:
features_train_up, features_test_up, target_train_up, target_test_up = train_test_split(dat_up.iloc[:,:-1], dat_up.iloc[:,-1], test_size=0.25)

In [ ]:
best_model_on_data_custom.fit(features_train_up, target_train_up)
test_predictions = best_model_on_data_custom.predict(features_test_up)
result = f1_score(target_test_up, test_predictions)
print("F1 лучшей модели", best_model_on_data_custom, "на тестовой выборке:", abs(round(result, 2)))

F1 лучшей модели RandomForestClassifier(max_depth=8, n_estimators=25) на тестовой выборке: 0.96


# Общий вывод

**I. В результате загрузки данных, установлено:**

1.  DataFrame содержит 159571 строк и 2 столбца.
2.  В столбце «text» данные типа object, пропуски отсутствуют.
3.  В столбце «toxic» данные типа int64, пропуски отсутствуют.
4.  Явные дубликаты отсутствуют.

**II. В результате предобработки:**

1. Произведена очистка данных и лемматизация (для BERT можно и не делать лемматизацию, она и так хорошо справляется).
2. Выбрано случайным образом 500 строк, ввиду ресурсоёмкости обработки всего массива данных.
3. Установлено, что соотношение класса '0' и '1' соответственно: 0.88 : 0.12, данный факт указывает на необходимость балансировки, однако, для начала необходимо подготовить модели.

**III. В результате подготовки модели, а также features & target:**

1.  Загружена предобученная модель `BertModel`.
2.  Выполнена токенизация и кодирование строк.
3.  Выполнена подрезка длинны токенов (не должно превышать 512 (обусловлено особенностями используемой модели BERT) 
4.  Выборка разделена на тренировочную и тестовую (75:25).

**IV. В результате обучения и подбора лучших параметров моделей на default данных, установлено:**

1. Лучшая модель: LogisticRegression.
2. Параметры лучшей модели: C=18, class_weight='balanced'.
3. Качество модели (F1): 0.444.
4. Не высокое качество модели обусловлено дисбалансом классов.

**V. В результате поиска множителя балансировки, а также подбора лучших параметров моделей на custom данных, установлено:**

1. Высокое качество модели RandomForestClassifier(max_depth=8, n_estimators=25) достигается на множителе - 4.
2. Значение F1: 0.86.
3. Время поиска множителя для балансировки и параметров модели порядка: 8 мин.

**VI. В результате проверки качества модели на тестовой выборке, установлено:**

1. для best_model_on_data_default
  * F1 лучшей модели LogisticRegression(C=18, class_weight='balanced') на тестовой выборке: 0.52;
  * Требование бизнеса (F1 не меньше 0.75) – не выполнено;
2. для best_model_on_data_custom
  * F1 лучшей модели RandomForestClassifier(max_depth=8, n_estimators=25) на тестовой выборке: 0.96;
  * Множитель, увеличивающий выборку - 4;
  * Требование бизнеса (F1 не меньше 0.75) - выполнено.